### Extract weights, biases, inputs, outputs of VGG16 into a pickled dictionary as float16

Tensorflow format:
* inputs/outputs - (N,H,W,C)
* kernel - (K,K,CIN,COUT)
* bias - (COUT,)


Dictionary format:

```
* inputs/outputs - (N,H,W,C)
* kernel         - (K,K,CIN,COUT)
* bias           - (COUT,)

* arrays_dict
    * 'layer_name':
        * 'input ': ndarray
        * 'output': ndarray
        * 'kernel': ndarray - if conv in layer_name
        * 'bias  ': ndarray - if conv in layer_name
```


In [21]:
import tensorflow as tf
import numpy as np
import pickle

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

tf.keras.backend.set_floatx('float16')

In [22]:
vgg16_model = tf.keras.applications.VGG16(include_top=True, weights='imagenet')

In [23]:
img = image.load_img('elephant.jpg', target_size=(224, 224))
x   = image.img_to_array(img)
x   = np.expand_dims(x, axis=0)
x   = preprocess_input(x)

out = vgg16_model.predict(x)

In [24]:
x.shape

(1, 224, 224, 3)

In [25]:
layer_names = [layer.name for layer in vgg16_model.layers]
layer_names

['input_2',
 'block1_conv1',
 'block1_conv2',
 'block1_pool',
 'block2_conv1',
 'block2_conv2',
 'block2_pool',
 'block3_conv1',
 'block3_conv2',
 'block3_conv3',
 'block3_pool',
 'block4_conv1',
 'block4_conv2',
 'block4_conv3',
 'block4_pool',
 'block5_conv1',
 'block5_conv2',
 'block5_conv3',
 'block5_pool',
 'flatten',
 'fc1',
 'fc2',
 'predictions']

In [26]:
arrays_dict = {}
config = {}
indices  = {}
tensors = []
input_tensor = None
i = 0

for name in layer_names:
    if 'input' in name:
        input_tensor = vgg16_model.get_layer(name).output
    else:
        layer = vgg16_model.get_layer(name)
        
        arrays_dict[name] = {}
        config[name] = {}
        config[name]['layer'] = layer
        
        if 'conv' in name:
            arrays_dict[name]['kernel'] = layer.weights[0].numpy()
            arrays_dict[name]['bias']   = layer.weights[1].numpy()
        
        tensors += [layer.input]
        config[name]['input_index']  = i
        i += 1
        
        tensors += [layer.output]
        config[name]['output_index'] = i
        i += 1
        
tmp_model = tf.keras.Model(inputs=input_tensor, outputs=tensors)
arrays = tmp_model.predict(x)

assert len(arrays) == i

for layer_name in arrays_dict:
    arrays_dict[layer_name]['input']  = arrays[config[layer_name]['input_index']]
    arrays_dict[layer_name]['output'] = arrays[config[layer_name]['output_index']]

In [27]:
arrays_dict['block1_conv1']['bias']

array([ 0.7344 ,  0.0934 ,  0.06775,  0.886  ,  0.26   ,  0.664  ,
       -0.01582,  0.325  ,  0.686  ,  0.06247,  0.5815 ,  0.2362 ,
        0.697  ,  0.1945 ,  0.4858 ,  0.4458 ,  0.511  ,  0.2086 ,
        0.5757 ,  0.332  ,  0.4998 ,  0.712  ,  0.3027 ,  0.7085 ,
        0.0455 ,  0.7446 ,  0.2983 ,  0.4822 ,  0.8164 ,  0.626  ,
        0.3767 ,  2.064  ,  0.773  ,  0.346  ,  0.6133 ,  0.655  ,
        0.3904 ,  0.509  ,  0.2625 ,  0.2367 ,  0.0797 ,  1.118  ,
        0.2612 ,  0.8696 ,  0.2155 ,  0.7803 ,  0.37   ,  0.48   ,
        0.4314 ,  0.5083 ,  0.2305 ,  0.7637 ,  0.3542 ,  0.458  ,
        0.4663 ,  0.0985 ,  0.3804 ,  0.669  ,  0.4016 ,  0.9053 ,
        0.4316 ,  1.302  ,  0.531  ,  0.49   ], dtype=float16)

In [28]:
with open('vgg16_dict.pickle', 'wb') as f:
    pickle.dump(arrays_dict, f)

In [29]:
arrays_dict_open = pickle.load(open('vgg16_dict.pickle', 'rb'))
arrays_dict_open['block1_conv1']['kernel'].shape

(3, 3, 3, 64)

In [30]:
list(arrays_dict.keys())

['block1_conv1',
 'block1_conv2',
 'block1_pool',
 'block2_conv1',
 'block2_conv2',
 'block2_pool',
 'block3_conv1',
 'block3_conv2',
 'block3_conv3',
 'block3_pool',
 'block4_conv1',
 'block4_conv2',
 'block4_conv3',
 'block4_pool',
 'block5_conv1',
 'block5_conv2',
 'block5_conv3',
 'block5_pool',
 'flatten',
 'fc1',
 'fc2',
 'predictions']

In [19]:
# PROBLEM - conv includes relu

In [33]:
vgg16_model.save('saved')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved\assets


In [1]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('saved')
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()

MemoryError: 